## Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da aula passada, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada.

Na camada de auto-atenção, deve-se implementar (vide slide 34):
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Camada de feed forward (2-layer MLP)

Instrucões:
- É necessário fazer duas implementações da camada de auto-atenção: uma usando laços (ineficiente, mas fácil de entender) e outra matricial (eficiente mas difícil de entender). Usar slide 36 como referência.

- Fazer um assert para garantir que o resultado das duas implementações é exatamente igual.

- No treinamento, usar apenas a implementação matricial.

In [1]:
import os
import sys
import random
import torch.nn as nn
import torch.nn.functional as F
import time
from sklearn.model_selection import train_test_split

In [2]:
# Global variables

# Vocabulary
vocab_size = 3000
context_size = 5
pattern = r'\w+|[,;.:!?\']'

# Training
batch_size = 32
epochs = 10
lr = 0.01

# Model
embedding_dim = 64
hidden_dim = 128

## Faz download e carrega o dataset

In [3]:
# Check if download is necessary
if not os.path.exists("67724.txt.utf-8"):
    print("Downloading Gutenberg texts")

    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [4]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")

len(paragraphs)

4969

In [5]:
# Checking the text
print(paragraphs[0])

The Project Gutenberg eBook of O Guarany: romance brazileiro, Vol. 1 (of 2)
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.


In [6]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

# Print 5 random paragraphs
num_paragraphs = len(cleaned_paragraphs)
for i in range(0,5):
    idx = random.randrange(num_paragraphs)
    print(f"{cleaned_paragraphs[idx]}\n")

print("Number of paragraphs: " + str(num_paragraphs))

len(cleaned_paragraphs)

--Espera, senhora!

O campo, esse tinha para ella outras recordações ainda vivas e palpitantes; a flor da sua mocidade tinha sido bafejada por essas auras; o botão desatára aos raios desse sol esplendido.

--Assim não te oppões?

Mas coragem é o que lhe faltava; por duas ou tres vezes o aventureiro teve um impeto de suspender-se ao frechai, e deixar a taboa rolar no abysmo; não passou de um desejo.

--Chama-me _Cecy_, dizia ás vezes ao indio sorrindo-se; este doce nome me lembrará que fui má para ti; e me ensinará a ser boa.

Number of paragraphs: 4892


4892

## Análise do dataset

In [7]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(pattern, text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12610

## Criando um vocabulário

In [8]:
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [9]:
print(f"Most Frequent Words: {most_frequent_words[:10]}")
print(f"Vocabulary Size: {len(vocab)}")

Most Frequent Words: ['.', ',', 'a', 'que', 'o', 'de', 'e', 'se', ';', 'um']
Vocabulary Size: 3000


In [10]:
#pattern = r'\w+'

def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(pattern, sentence.lower())]

print(cleaned_paragraphs[20])
print(encode_sentence(cleaned_paragraphs[20], vocab))

 Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
[0, 146, 0, 23, 0, 2, 8, 50, 117, 276, 266, 2669, 13, 1071, 0, 2, 4, 193, 137, 287, 5, 2264, 8, 0, 3, 2672, 1]


## Classe do dataset

In [11]:
# Dataset class
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, paragraphs, vocab, context):
    self.paragraphs = paragraphs
    self.vocab = vocab
    self.context = context
    self.tokens, self.targets = self.setup()

  def __len__(self):
    return len(self.tokens)

  def __getitem__(self, idx):
    return torch.tensor(self.tokens[idx]), torch.tensor(self.targets[idx])
  
  def setup(self):
    tokens = []
    targets = []
    for paragraph in self.paragraphs:
      encoded = encode_sentence(paragraph, self.vocab)
      
      # If paragraph is smaller than the context, skip it.
      if len(encoded) < self.context + 1:
          continue

      for i in range(len(encoded) - self.context):
        tks = encoded[i:i+self.context]
        tgt = encoded[i+self.context]
        # Only add if there are no unknown tokens in both context and target.
        bad_token = 0
        if not (bad_token in tks or tgt == bad_token):
          tokens.append(tks)
          targets.append(tgt)
    return tokens, targets


In [12]:
# Train/Validation split
train_data, val_data = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

train_dataset = CustomDataset(train_data, vocab, context_size)
val_dataset = CustomDataset(val_data, vocab, context_size)

# Counting all Samples
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print()
print(f"Training dataset samples: {len(train_dataset)}")
print(f"Validation dataset samples: {len(val_dataset)}")

Training samples: 3913
Validation samples: 979

Training dataset samples: 44547
Validation dataset samples: 12257


In [13]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

sample = next(iter(train_loader))
print(sample)

[tensor([[2604,  581,    4,  142,    8],
        [ 354,    6,   17,  860,    3],
        [   5,   80,    2,   16,    5],
        [   4, 1578,    3,  118,   83],
        [   2,    7, 1872,   27,  399],
        [ 144,    6,   19,  167,    2],
        [   3,   99,    5,  215,    6],
        [ 119,    2,  237,   82,   79],
        [ 181,    9,   89,    5,  351],
        [   1,   49,    1,    1,    1],
        [  50,    5,   43,  910,    9],
        [  30,    1,   47,    6,   73],
        [   7,  105, 1146,    1,    1],
        [  54,  285,    3,   99, 1115],
        [1042,    7, 1618,    2,    3],
        [  44,   70,   25,  185,   49],
        [ 140,    1,    7,    1, 1931],
        [  29,   16,   10,  458, 1694],
        [ 665,   11,  107,    9,   62],
        [   2, 1233,    3,  118,  943],
        [   2,    7,  903,   13,  671],
        [1181,    1,   33,  146,   10],
        [  14, 1296,    1,  111,    3],
        [   6,   19,  167,    2,  103],
        [   1,    5,  119,  495,    4],

## Model

In [14]:
class BengioModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.linear2 = nn.Linear(h, vocab_size+1)
        self.relu = torch.nn.ReLU()
        self.tanh = torch.nn.Tanh()

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        #out = self.relu(out)
        out = self.tanh(out)
        # Second layer
        out = self.linear2(out)
        return out

In [15]:
model = BengioModel(vocab_size, embedding_dim, context_size, hidden_dim)

In [16]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

print(input.shape)
print(target.shape)

torch.Size([32, 5])
torch.Size([32])


In [17]:
output = model(input)

In [18]:
output.argmax(dim=1)

tensor([2123,  371, 2848, 2666, 1153, 1802,  183, 2342, 1344,  531,   28, 2147,
         128, 2453,  470,  533, 2807,  921, 2343, 1639, 2453,  435,  913,  280,
        2145, 1862, 2745,  379, 2975,  219,  776, 1589])

In [19]:
target

tensor([   1,  456, 2254,   17,   18,    5, 1157,  191,    7, 1653,   79, 2158,
         100, 2312,   29,   83,   36,    9,  290,   31,  629,  116,  311,    6,
         389,   46,  113,  211,   24,    8,   16, 1620])

## Training

In [20]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [21]:
# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr)

model.to(device)

BengioModel(
  (embeddings): Embedding(3001, 64)
  (linear1): Linear(in_features=320, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=3001, bias=True)
  (relu): ReLU()
  (tanh): Tanh()
)

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Exemplo de uso:
total_params = count_parameters(model)
print(f'O modelo tem um total de {total_params:,} parâmetros.')

O modelo tem um total de 620,281 parâmetros.


In [23]:
# Initial Perplexity and Loss
# Before training
model.eval()

loss = 0
perp = 0

with torch.no_grad():
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        loss += criterion(outputs, targets).item()

loss /= len(train_loader)
perp = torch.exp(torch.tensor(loss))

print(f'Initial Loss: {loss:.4f}')
print(f'Initial Perplexity: {perp:.4f}')

Initial Loss: 8.0418
Initial Perplexity: 3108.3467


In [24]:
# Training Loop
model.train()
for epoch in range(epochs):

  epoch_start = time.time()
  # Metrics
  epoch_loss = 0
  epoch_correct = 0
  epoch_samples = 0

  for inputs, targets in train_loader:
        inputs = inputs.to(device)  # Move input data to the device
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Loss
        epoch_loss += loss.item()

        # Predicted
        _, predicted = torch.max(outputs, 1)
        epoch_correct += (predicted == targets).sum().item()
        epoch_samples += targets.size(0)

  # Calculate average loss and accuracy for epoch
  avg_loss = epoch_loss / len(train_loader)
  acc = epoch_correct / epoch_samples

  # Perplexity
  perp = torch.exp(torch.tensor(avg_loss))

  epoch_end = time.time()
  epoch_time = epoch_end - epoch_start
  # Print epoch statistics
  print(f'Epoch [{epoch+1}/{epochs}], Time:{epoch_time:.2f}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Perplexity: {perp:.4f}')


Epoch [1/10], Time:5.34, Loss: 6.3265, Accuracy: 0.11%, Perplexity: 559.1981
Epoch [2/10], Time:5.28, Loss: 5.5142, Accuracy: 0.15%, Perplexity: 248.1925
Epoch [3/10], Time:5.15, Loss: 4.9896, Accuracy: 0.18%, Perplexity: 146.8778
Epoch [4/10], Time:5.12, Loss: 4.6371, Accuracy: 0.21%, Perplexity: 103.2423
Epoch [5/10], Time:5.11, Loss: 4.3744, Accuracy: 0.23%, Perplexity: 79.3898
Epoch [6/10], Time:5.41, Loss: 4.2042, Accuracy: 0.26%, Perplexity: 66.9701
Epoch [7/10], Time:5.53, Loss: 4.0587, Accuracy: 0.27%, Perplexity: 57.9012
Epoch [8/10], Time:5.52, Loss: 3.9619, Accuracy: 0.29%, Perplexity: 52.5579
Epoch [9/10], Time:5.53, Loss: 3.8790, Accuracy: 0.30%, Perplexity: 48.3780
Epoch [10/10], Time:5.50, Loss: 3.8114, Accuracy: 0.31%, Perplexity: 45.2141


## Avaliação

In [25]:
model.eval()

loss_sum = 0
total_sum = 0
correct_sum = 0
eval_round = 0

with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)        
        loss_sum += loss

        # Get the predicted labels
        _, predicted = torch.max(outputs, 1)

        total_sum += targets.size(0)
        correct_sum += (predicted == targets).sum().item()
        eval_round += 1

# Calculate accuracy
acc = 100 * correct_sum/total_sum

# Calculate average perplexity
average_loss = loss_sum / eval_round
average_perplexity = torch.exp(average_loss)

print(f'Test Accuracy: {acc:.2f}%')
print(f'Average Loss: {average_loss:.2f}')
print(f'Average Perplexity: {average_perplexity:.2f}')

Test Accuracy: 18.96%
Average Loss: 9.61
Average Perplexity: 14929.84


## Exemplo de uso

In [26]:
# Código adaptado da implementação do Cesar Bastos
from colorama import Fore, Style

text = cleaned_paragraphs
model.to(device)
def generate_text(model, vocab, text, max_length, context_size):
    words = []
    # Ensure there are enough words for at least one sequence
    while len(words) < context_size:
        random_number = random.randint(1, 4891)
        words = encode_sentence(text[random_number], vocab)
        if not words:
            words = []
            continue  # Skip if the sentence cannot be encoded
        words = words[:context_size]
        #print(words)
        if any(token == 0 for token in words):
            words = []
            continue  # Skip if any token is zero (assuming 0 is a special token)
        context = words

    print(f"Frase: {cleaned_paragraphs[random_number]}")
    print(words)

    for _ in range(max_length):
        words_tensor = torch.tensor(context[-context_size:], dtype=torch.long).unsqueeze(0).to(device)
        logits = model(words_tensor)
        probs = F.softmax(logits, dim=1)
        next_token = torch.multinomial(probs, num_samples=1)
        context.append(next_token.item())
        print(context)
    frase = []
    for i in context: ##Agradecimentos a Ramon Abilio
        word = next((word for word, code in vocab.items() if code == i), "<UNKNOWN>")
        frase.append(word)

    print(f"{Fore.BLUE}{frase[:context_size]}{Style.RESET_ALL} {Fore.RED}{frase[-max_length:]}{Style.RESET_ALL} ")


max_length= 10
generate_text(model, vocab, text, max_length, context_size)

Frase: Pery, tendo satisfeito o desejo da moça, retirou-se e voltou ao seu trabalho, que elle proseguia com uma constancia infatigavel.
[21, 2, 677, 1300, 5]
[21, 2, 677, 1300, 5, 186]
[21, 2, 677, 1300, 5, 186, 1485]
[21, 2, 677, 1300, 5, 186, 1485, 1024]
[21, 2, 677, 1300, 5, 186, 1485, 1024, 12]
[21, 2, 677, 1300, 5, 186, 1485, 1024, 12, 56]
[21, 2, 677, 1300, 5, 186, 1485, 1024, 12, 56, 1418]
[21, 2, 677, 1300, 5, 186, 1485, 1024, 12, 56, 1418, 3]
[21, 2, 677, 1300, 5, 186, 1485, 1024, 12, 56, 1418, 3, 90]
[21, 2, 677, 1300, 5, 186, 1485, 1024, 12, 56, 1418, 3, 90, 511]
[21, 2, 677, 1300, 5, 186, 1485, 1024, 12, 56, 1418, 3, 90, 511, 42]
['pery', ',', 'tendo', 'satisfeito', 'o'] ['sol', 'chamar', 'naturalmente', 'não', 'te', 'entendo', 'a', 'or', 'vista', 'dos'] 
